In [1]:
# libraries 
import numpy as np
import pandas as pd
import requests
! pip install XlsxWriter
import xlsxwriter
import math


     |████████████████████████████████| 153kB 5.6MB/s 


In [2]:
stocks = pd.read_csv("/content/sp_500_stocks.csv")
stocks.tail()

,Ticker
500,YUM
501,ZBH
502,ZBRA
503,ZION
504,ZTS


In [3]:
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'



In [4]:
symbol = "AAPL"
api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(api_url).json()
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'SNAETQ(RMAAL/LE SKTD NCO)GESLAG B', 'calculationPrice': 'close', 'open': 138.14, 'openTime': 1660702090578, 'openSource': 'faifoilc', 'close': 140.2, 'closeTime': 1627503752189, 'closeSource': 'cfioialf', 'high': 143.48, 'highTime': 1682226780556, 'highSource': 'te5 mceildruy 1nde iape', 'low': 137.05, 'lowTime': 1629857063897, 'lowSource': ' ere5y dimp ecdentiau1l', 'latestPrice': 138.74, 'latestSource': 'Close', 'latestTime': 'January 21, 2021', 'latestUpdate': 1647888417486, 'latestVolume': 123332775, 'iexRealtimePrice': 138.45, 'iexRealtimeSize': 103, 'iexLastUpdated': 1691295575950, 'delayedPrice': 143.14, 'delayedPriceTime': 1641583469180, 'oddLotDelayedPrice': 137, 'oddLotDelayedPriceTime': 1680816576864, 'extendedPrice': 139.18, 'extendedChange': -0.49, 'extendedChangePercent': -0.00362, 'extendedPriceTime': 1675716698859, 'previousClose': 136.53, 'previousVolume': 107696549, 'change': 5.04, 'changePercent': 0.0

In [5]:
price = data["latestPrice"]
market_cap = data["marketCap"]

In [6]:
my_columns = ['Ticker','Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe.head()

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [7]:
final_dataframe.append(
    pd.Series(
        [
          symbol,
         price,
         market_cap,
         'N/A'
        ], index = my_columns
    ), ignore_index = True
)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,138.74,2338020268208,N/A


In [8]:
# Looping through the tickers in our list of stocks
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker']:
  api_url = f"https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}"
  data  = requests.get(api_url).json()
  final_dataframe  = final_dataframe.append(
      pd.Series(
          [
           stock,
           data["latestPrice"],
           data["marketCap"],
           'N/A'
          ], index = my_columns
    ), ignore_index = True
)

In [9]:
final_dataframe.tail()

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
500,YUM,111.03,33945338608,N/A
501,ZBH,165.3,35358358062,N/A
502,ZBRA,415.9,22703155838,N/A
503,ZION,49.49,8012105227,N/A
504,ZTS,170.55,78417497604,N/A


In [10]:
def chunks(lst, n):
  for i in range(0, len(lst), n):
    yield lst[i:i + n]

In [11]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
  symbol_strings.append(','.join(symbol_groups[i]))
  #print(symbol_strings[i])

final_dataframe = pd.DataFrame(columns = my_columns)

for string in symbol_strings:
  batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={string}&types=quote,quote&token={IEX_CLOUD_API_TOKEN}'
  data = requests.get(batch_api_call_url).json()
  for symbol in string.split(','):
    final_dataframe = final_dataframe.append(
        pd.Series(
            [
             symbol,
             data[symbol]['quote']['latestPrice'],
             data[symbol]['quote']['marketCap'],
             'N/A'
            ], index = my_columns
        ), ignore_index = True
    )

In [12]:
final_dataframe.tail()

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
500,YUM,109.99,32601298963,N/A
501,ZBH,169.16,33883517715,N/A
502,ZBRA,409.7,22665242081,N/A
503,ZION,47.98,8114160017,N/A
504,ZTS,169.93,79223601667,N/A


In [18]:
portfolio_size = input('Enter the value of your portfolio')

val = float(portfolio_size)

Enter the value of your portfolio1000000


In [26]:
position_size = val/len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
  final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Stock Price'])

In [27]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,132,39989792764,15
1,AAL,16.28,9964525629,121
2,AAP,169.53,11720653302,11
3,AAPL,141.12,2320437205944,14
4,ABBV,115.14,203840315771,17
...,...,...,...,...
500,YUM,109.99,32601298963,18
501,ZBH,169.16,33883517715,11
502,ZBRA,409.7,22665242081,4
503,ZION,47.98,8114160017,41


In [28]:
# Formating an excel output
writer = pd.ExcelWriter('recommended trades.xlsx', engine = 'xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index = False)

In [29]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {   
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [25]:
#writer.sheets['Recommended Trades'].set_column('A:A',18,string_format)
#writer.sheets['Recommended Trades'].set_column('B:B',18,string_format)
#writer.sheets['Recommended Trades'].set_column('C:C',18,string_format)
#writer.sheets['Recommended Trades'].set_column('D:D',18,string_format)
#writer.save()

In [30]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', integer_format],
}
for column in column_formats.keys():
  writer.sheets['Recommended Trades'].set_column(f'{column}:{column}',18,column_formats[column][1])

writer.save()